In [13]:
%cd C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor

import sys
from pathlib import Path
from datetime import date

from src.pipeline.create_training_dataset import *

import xarray as xr
from tqdm import tqdm

import yaml
from pathlib import Path
import logging

from src.utils.grid_utils import Grid
import src.data_src.point_data_sources as pds
import src.data_src.gridded_data_sources as gds
from src.utils.data_utils import DataMapping

C:\Users\ehate\Desktop\ASP\ASP_code_projects\ASP_predictor


In [7]:
logger.info("Starting dataset creation...")

#project_root = Path(__file__).resolve().parents[2]
project_root = Path.cwd()
config_path = project_root / "configs/pipeline_configs/create_training_dataset.yaml"
config = yaml.safe_load(open(config_path, "r"))

target_grid = Grid.from_predefined(config['grid'])

# %% make point data dataframe
# Load point data sources
logger.info("Loading point data sources...")
point_sources = []
for source_cfg in tqdm(config['point_data_sources'], desc="Loading point data sources"):
    for sid_cfg in config['point_data_sources'][source_cfg]['secondary_id']:
        source = load_point_data_source(
            filepath=config['point_data_sources'][source_cfg]['secondary_id'][sid_cfg]['filepath'],
            secondary_id=config['point_data_sources'][source_cfg]['secondary_id'][sid_cfg]['id']
        )
        point_sources.append(source)

# Grid and merge point data sources
logger.info("Gridding and merging point data sources...")
gridded_point_data = grid_and_merge_point_data_sources(point_sources, target_grid)

# %% match to gridded data sources

# # CETB
# logger.info("Matching to CETB data...")
# CETB_mapping = DataMapping(config['CETB']['directory'], dataset='CETB')
# for i, channel in enumerate(config['CETB']['channels']):
#     logger.info(f"Processing CETB channel: {channel}" + f" ({i+1}/{len(config['CETB']['channels'])})")
#     files = CETB_mapping.get_by_channel(channel)['filename'].tolist()
#     scene = gds.CETBScene.from_files(files)
#     match = sample_from_CETB(gridded_point_data, scene)
#     gridded_point_data.data[channel] = match.data[channel]

# ERA5
logger.info("Matching to ERA5 data...")
ERA5_mapping = DataMapping(config['ERA5']['directory'], dataset='ERA5')
for i, channel in enumerate(config['ERA5']['channels']):
    logger.info(f"Processing ERA5 channel:{channel}" + f" ({i+1}/{len(config['ERA5']['channels'])})")
    files = ERA5_mapping.get_by_channel(channel)['filename'].tolist()
    scene = gds.ERA5Scene.from_files(files)
    match = sample_from_ERA5(gridded_point_data, scene)
    gridded_point_data.data[channel] = match.data[channel]

logger.info("saving full dataset...")
full_dataset = gridded_point_data
full_dataset.to_parquet(config['full_dataset']['directory'] / config['full_dataset']['name'])

INFO:src.pipeline.create_training_dataset:Starting dataset creation...
INFO:src.pipeline.create_training_dataset:Loading point data sources...
Loading point data sources: 100%|██████████| 3/3 [00:49<00:00, 16.62s/it]
INFO:src.pipeline.create_training_dataset:Gridding and merging point data sources...
Processing dates: 100%|██████████| 4/4 [00:06<00:00,  1.62s/date]
INFO:src.pipeline.create_training_dataset:Matching to ERA5 data...
INFO:src.pipeline.create_training_dataset:Processing ERA5 channel:10u (1/20)


KeyError: '10u'

In [9]:
match.data

,primary_id,secondary_id,time,doy,x,y,row,col,lon,lat,...,cluster_distance_to_cell_center,SD_mean,SD_median,SD_min,SD_max,SD_std,SD_MAD,SD_SE,SD_obs_unc,u10
0,C2I,2020-2021,2020-11-02 12:35:52.732885504,307,-626562.5,223437.5,2808,2679,-109.626665,84.041447,...,1987.925071,0.203202,0.203202,0.191121,0.215282,0.017085,0.012081,0.008542,0.089900,1.814439
1,C2I,2020-2021,2020-11-02 12:35:52.944098304,307,-623437.5,223437.5,2808,2680,-109.717476,84.067831,...,1611.690504,0.216314,0.215653,0.208157,0.222914,0.006218,0.006335,0.002487,0.089900,1.836577
2,C2I,2020-2021,2020-11-02 12:35:53.339560960,307,-620312.5,223437.5,2808,2681,-109.809097,84.094199,...,1772.924901,0.205157,0.203924,0.198098,0.214772,0.007304,0.005826,0.002922,0.089900,1.858651
3,C2I,2020-2021,2020-11-02 12:35:54.036115456,307,-617187.5,223437.5,2808,2682,-109.901538,84.120552,...,1599.802439,0.258431,0.259452,0.252720,0.263260,0.004415,0.003198,0.001645,0.089900,1.880620
4,C2I,2020-2021,2020-11-02 12:35:54.334638592,307,-614062.5,223437.5,2808,2683,-109.994811,84.146889,...,1512.302830,0.237062,0.238703,0.210088,0.257578,0.016349,0.012836,0.005721,0.089900,1.902524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124118,AEM_AWI,PAMARCMIP,2017-04-04 19:54:23.856000512,94,-1289062.5,1526562.5,2391,2467,-139.821527,72.032362,...,904.929720,0.150830,0.147000,0.006000,0.584000,0.108122,0.070000,0.000848,0.025369,-9.433546
124119,AEM_AWI,PAMARCMIP,2017-04-04 19:54:54.701182208,94,-1285937.5,1526562.5,2391,2468,-139.890064,72.050642,...,2010.190552,0.269231,0.239000,0.084000,0.710000,0.155282,0.088000,0.003376,0.024740,-9.453643
124120,AEM_AWI,PAMARCMIP,2017-04-04 19:55:23.368170496,94,-1285937.5,1523437.5,2392,2468,-139.832203,72.072316,...,1107.253394,0.210938,0.175000,0.010000,0.741000,0.138390,0.076000,0.001141,0.025924,-9.417876
124121,AEM_AWI,PAMARCMIP,2017-04-04 19:56:15.648478464,94,-1282812.5,1523437.5,2392,2469,-139.900902,72.090590,...,47.363430,0.135234,0.113000,0.009000,0.598000,0.105770,0.057000,0.000800,0.025543,-9.436493


In [15]:
logger.info("Starting dataset creation...")

#project_root = Path(__file__).resolve().parents[2]
project_root = Path.cwd()
config_path = project_root / "configs/pipeline_configs/create_training_dataset.yaml"
config = yaml.safe_load(open(config_path, "r"))


import pandas as pd
df = pd.read_parquet(Path(config['datasets']['full']['save_directory']) / Path(config['datasets']['full']['name']))
df.columns

INFO:src.pipeline.create_training_dataset:Starting dataset creation...


Index(['primary_id', 'secondary_id', 'time', 'doy', 'x', 'y', 'row', 'col',
       'lon', 'lat', 'num_samples', 'samples_mean_lon', 'samples_mean_lat',
       'samples_mean_x', 'samples_mean_y', 'mean_distance_to_cell_center',
       'cluster_distance_to_cell_center', 'SD_mean', 'SD_median', 'SD_min',
       'SD_max', 'SD_std', 'SD_MAD', 'SD_SE', 'SD_obs_unc', '6.9H', '6.9V',
       '10.7H', '10.7V', '18H', '18V', '23H', '23V', '36H', '36V', '89H',
       '89V', 'u10', 'v10', 'd2m', 't2m', 'tp', 'sst', 'skt', 'fal', 'siconc',
       'tcwv', 'ssrd', 'strd', 'tclw', 'tciw', 'sf', 'e', 'istl1', 'istl2',
       'istl3', 'istl4'],
      dtype='object')